## Classification benchmarks: `Zero-Shot-Learner`, `Few-Shot-Learner`, `BERT finetuning`, `GPT finetuning`, `GPT prompting`  

GPT models can also be used for classification tasks. Here I'm putting together a classification benchmark by comparing a ZSL, an FSL, a finetuned BERT and finetuned / prompted GPT models to see how they perform.

OpenAI docs: 
- https://platform.openai.com/docs/guides/fine-tuning
- https://platform.openai.com/docs/guides/fine-tuning/advanced-usage
- https://github.com/openai/openai-cookbook/blob/main/examples/Fine-tuned_classification.ipynb

In [ ]:
import credentials
import os
os.environ["OPENAI_API_KEY"] = credentials.openai_api

import json
import openai